In [1]:
import pandas as pd
from collections import Counter
import numpy as np
import random
import csv

from pandarallel import pandarallel

pandarallel.initialize(progress_bar=False)


processed_dir = '/home/libbyh/belfer-dtm/data/'
browser_data_dir = '/home/libbyh/belfer-dfrbrowser/data/'

INFO: Pandarallel will run on 36 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
filename = processed_dir + 'stormfront_post_data_processed.json.gz'

df = pd.read_json(filename, orient = "index")
df.reset_index(inplace=True)
df.head()


,index,url,post_date,post_text,post_author,thread_forum,yearmonth,year,quarter,processed,processed_common
0,195938,https://www.stormfront.org/forum/t899963-3/,"08-19-2012, 07:00 PM",Quote:\n\n\n\n\nOriginally Posted by Slarioux\...,SS Sturmbrigade Langemarck,Revisionisme,"{'day': 31, 'dayofyear': 244, 'daysinmonth': 3...","{'day': 31, 'dayofyear': 366, 'daysinmonth': 3...","{'day': 30, 'dayofyear': 274, 'daysinmonth': 3...",slarioux day dat opzicht niet noodzakelijk mee...,day dat niet meer kunnen wel men met day dat w...
1,277294,https://www.stormfront.org/forum/t1193139-6/#g...,"01-01-2017, 04:26 PM",Quote:\n\n\n\n\nOriginally Posted by Gary Hans...,James Harting,The Eternal Flame,"{'day': 31, 'dayofyear': 31, 'daysinmonth': 31...","{'day': 31, 'dayofyear': 365, 'daysinmonth': 3...","{'day': 31, 'dayofyear': 90, 'daysinmonth': 31...",gary hansen heard different version killed fir...,heard different version killed first would lik...
2,95329,https://www.stormfront.org/forum/t273502-2/,"06-22-2006, 10:11 AM",Quote:\n\n\n\n\nOriginally Posted by lukaskins...,Milan Stojadinovic mladji,Женски кутак,"{'day': 30, 'dayofyear': 181, 'daysinmonth': 3...","{'day': 31, 'dayofyear': 365, 'daysinmonth': 3...","{'day': 30, 'dayofyear': 181, 'daysinmonth': 3...",lukaskins hosimin,
3,261263,https://www.stormfront.org/forum/t1311829-2/?#,"07-01-2020, 04:04 AM",I don't really know what to think about this t...,BreadandCircuses,Lounge,"{'day': 31, 'dayofyear': 213, 'daysinmonth': 3...","{'day': 31, 'dayofyear': 366, 'daysinmonth': 3...","{'day': 30, 'dayofyear': 274, 'daysinmonth': 3...",really know think thread bit late making allow...,really know think thread bit late making possi...
4,126832,https://www.stormfront.org/forum/t1309146/,"06-02-2020, 08:22 PM","Greetings,\n\nI'm a 43 y/o white male who is o...",Pythonis Rogue,New Members Introduce Yourselves,"{'day': 30, 'dayofyear': 182, 'daysinmonth': 3...","{'day': 31, 'dayofyear': 366, 'daysinmonth': 3...","{'day': 30, 'dayofyear': 182, 'daysinmonth': 3...",greetings white male california usa call midwe...,greetings white male california usa call regio...


In [3]:
# optionally sample for testing
df = df.sample(n=25000, replace=False, random_state=1)
print(len(df))

25


In [4]:
#reformat metadata, create dataframe, write to csv
column_names = ["id", "title", "author", "journal", "volume", "issue", "pubdate", "pages"]
meta = df[['index', 'processed', 'post_author', 'thread_forum', 'post_date']]
meta.rename(columns={'index': 'id', 'processed': 'title', 'post_author': 'author', 'thread_forum': 'journal',
                    'post_date': 'pubdate'}, inplace=True)
meta['volume'] = ''
meta['issue'] = ''
meta['pages'] = ''

meta.set_index("id", inplace=True)

print(meta.head())

                                                    title       author  \
id                                                                       
228615  wingless dragon know still majority white like...          Roo   
93004   jewish congress hails austrian move ban wwii b...     revision   
267956  anarchy4 look islam wants get hands look whate...     ANARCHY4   
284424                                        upsets life  Der_Söldner   
51094   yes well border patrol effective kind screws t...     Strasser   

                       journal               pubdate volume issue pages  
id                                                                       
228615    Newslinks & Articles  06-28-2020, 07:39 AM                     
93004            European News  06-03-2020, 12:37 AM                     
267956     Culture and Customs  10-30-2008, 09:13 AM                     
284424  Nature and Environment  02-02-2013, 03:09 PM                     
51094     Newslinks & Articles  10-17

/sw/arcts/centos7/python3.7-anaconda/2020.02/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/sw/arcts/centos7/python3.7-anaconda/2020.02/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/sw/arcts/centos7/python3.7-anaconda/2020.02/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [5]:
meta.to_csv(browser_data_dir + 'sf_meta.csv', 
    sep=',', 
    quoting = csv.QUOTE_ALL, 
    encoding = 'utf-8')

In [6]:
# generate word frequency files for each post
i = 0

freqs = meta['title'].str.split().parallel_apply(pd.value_counts)

freqs = freqs.fillna(0)
freqs = freqs.astype('int8')

for index, row in freqs.iterrows():
    if i > len(df):
        break
    else:
        filename = browser_data_dir + 'freq/' + str(index) + '.tsv'
        row.dropna(inplace=True)
    with open(filename, 'w') as f:
        f.write(row.to_csv(sep='\t'))
    i+=1